# Generate Kinematic Models of Assemblies

Timor is built to generate kinematic and dynamic models of arbitrary configurations of robot modules (so-called assemblies). In this tutorial, we will first show how these models can be created using the builtin capabilities.

Secondly, we are going to conduct a short timing **experiment**: How long does it take to generate a large number of models? More precisely, we are replicating some of the work done in the paper [Effortless creation of safe robots from modules through self-programming and self-verification](https://doi.org/10.1126/scirobotics.aaw1924). There, the authors define a module set called "IMPROV" which is builtin with Timor, and eliminate unfit configurations in a hierarchical procedure. We are going to time how long it takes Timor to generate all configurations proposed in this work.

In [ ]:
from pathlib import Path

from timor.Module import ModuleAssembly, ModulesDB
from timor.utilities.file_locations import get_module_db_files
from timor.utilities import logging

The module set is builtin with timor - the data is [publicly available ](https://bitbucket.org/MatthiasAlthoff/improv/src/master) and has been transferred to the JSON format used by Timor by the authors of the toolbox.

In [ ]:
logging.setLevel('WARN')
db = ModulesDB.from_file(*get_module_db_files('IMPROV'))  # Note that currently, there is no support for compressed .wrl files - however, we can use the .stl files provided

### Generate a model
To define a robot, one can specify modules and connections used for the assembly as [described in the documentation](https://timor-python.readthedocs.io/en/latest/autoapi/timor/index.html#timor.ModuleAssembly).
For the sake of showcasing model generation, a random assembly serves our purpose just as well: Here, we assembly 13 modules -- twelve for which we make sure they are no end-effector, and a last one, which certainly is -- in a serial chain.

In [ ]:
assembly = ModuleAssembly(db)
for _ in range(12):
    assembly.add_random_from_db(lambda x: not any(c.type == 'eef' for c in x.available_connectors.values()))  # 12 non-end-effector modules
assembly.add_random_from_db(lambda x: any(c.type == 'eef' for c in x.available_connectors.values()))  # one module with an eef connector

### Get your robot model
You can directly transfer the assembly to a "PinRobot" - a robot representation based on the [Pinocchio library](https://github.com/stack-of-tasks/pinocchio) for rigid body dynamic algorithms.

It comes with useful capabilities, such as methods for forward and inverse kinematics, self-collision checks and visualization.

In [ ]:
robot = assembly.to_pin_robot()
robot.update_configuration(robot.random_configuration())
eef_position = robot.fk()
print("The relative transformation from base frame to end-effector frame is:", eef_position.homogeneous, sep='\n')

### Get the URDF
The [Unified Robot Description Format](http://wiki.ros.org/urdf) is a de-facto standard for describing manipulators. You can export any assembly with an open-chain kinematic (other are not natively supported by URDF) to the XML-based format and continue working with your robot model in any other simulation suite:

In [ ]:
urdf = assembly.to_urdf(name='My First Timor Robot')
print('\n'.join(urdf.split('\n')[:25]))  # Show the first 25 lines only to avoid cluttering the Notebook

# Experiment 1: How long does it take Timor to generate models?
For replicating the work of Althoff et al. from their 2019 paper ([open acccess link](https://mediatum.ub.tum.de/doc/1506779/gy8gyea2c5vyn65ely4bnzom6.Althoff-2019-ScienceRobotics.pdf)), we provide an iterator which yields only robot assemblies that are valid based on the filter criteria described in the sections "Search space reduction" and "Step-by-step elimination".

It is included in the "confguration search" module that implements lots of useful algorithms and helper methods for the search for an optimal assembly, given a task. We provide the following arguments as described in the paper:
- We are looking for a with exactly 6 degrees of freedom
- There can be at most 3 static "links" in between two joints
- The first joint is placed directly after the base (which is the default, but could be changed)
- After the last joint and before the end-effector, there can be 1 static extension (this is not mentioned in the paper, but in the supplementary material)

In [ ]:
from time import time
from timor.configuration_search.LiuIterator import Science2019 as Science2019Iterator
improv_iter = Science2019Iterator(db, min_dof=6, max_dof=6, max_links_between_joints=3, max_links_after_last_joint=1)  # This is the iterator
print("There are {} different models this iterator can yield".format(len(improv_iter)))

Taking a close look at the paper and the supplemetary code provided by Althoff et al., you will realize this number is too high! In fact, there are some limitations they mention in their paper and their code that are not yet implemented in the importet iterator, so let's add them:

In [ ]:
# First of all, the authors provide additional modules that were not used in the paper - we will eliminate them aswell
not_in_paper = ('L7', 'L10', 'PB22', 'PB24')
for module_name in not_in_paper:
    db.remove(db.by_name[module_name])
improv_iter = Science2019Iterator(db, min_dof=6, max_dof=6, max_links_between_joints=3, max_links_after_last_joint=1)  # Re-initialize the iterator for the filtered database

improv_iter.joint_combinations = (('21', '21', '23'), ('21', '21', '21'))  # There are hardware limitations about which joints can be used in which order

real_links = {'4', '5', '14', '15'}  # They make a distinction between "links" and "extensions" - there is at most one link between two joints, but up to two additional extensions
def paper_compliant_combo(links: tuple[str]) -> bool:
    """We encode our knowledge about valid links and extensions in a filter method"""
    is_link = tuple(link for link in links if link in real_links)
    if len(is_link) != 1:
        return False
    if len(links) == 3:
        if links[1] not in real_links:
            return False
    return True
# Now we update the "possible link combinations" as described in the paper
link_combinations_in_paper = tuple(link_combination for link_combination in improv_iter.link_combinations
                                   if paper_compliant_combo(link_combination))
improv_iter.link_combinations = link_combinations_in_paper

# We have to re-do some of the initialization of the iterator class to make the changes work
improv_iter.links_between_joints = improv_iter._identify_links_between_joints()
improv_iter.augmented_end_effectors = tuple(aeef for aeef in improv_iter.augmented_end_effectors if not any(
    real_link in aeef for real_link in real_links))  # Only extensions are allowed before the end effector, not all links
improv_iter.joint_combinations_for_base_eef = improv_iter._identify_base_chain_eef()
improv_iter._current_joint_combination = improv_iter.joint_combinations[improv_iter.state.joints]
improv_iter._current_num_joint_combinations = len(improv_iter.valid_joint_combinations)
improv_iter._current_num_joint_modules = len(improv_iter.joint_combinations[improv_iter.state.joints])

print("Now, there are {} different models the iterator can yield".format(len(improv_iter)))

This is exactly the number of combinations we expected. (You will also see that this piece of code is a duplicate of a unittest that led the development of this Iterator class -- an artifact of the test-driven-development that is one of the main coding principles of Timor)

In [ ]:
t0 = time()
for i, assembly in enumerate(improv_iter):
    urdf = assembly.to_urdf()
t1 = time()

delta_t = t1 - t0
print(f'It took {delta_t:.2f}s to generate {i+1} urdf models. This equals {1000 * (delta_t / (i+1)):.2f}ms/model')